In [25]:
!pip install ale_py

import gymnasium as gym
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv
import numpy as np
import ale_py

from stable_baselines3.common.env_util import make_vec_env, make_atari_env
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv, VecFrameStack
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.evaluation import evaluate_policy
import matplotlib.pyplot as plt
from gymnasium.wrappers import RecordVideo
import torch
import torch.nn as nn
from tqdm import tqdm
%matplotlib inline
from gymnasium.wrappers import AtariPreprocessing

In [26]:
TRAIN_STEPS = 2000000
EVAL_EPISODES = 10

In [29]:
gym.register_envs(ale_py)

train_env = make_atari_env('ALE/Pacman-v5', n_envs=16, seed=0)
train_env = VecFrameStack(train_env, n_stack=4)

eval_env = make_atari_env('ALE/Pacman-v5', n_envs=1, seed=0)
eval_env = VecFrameStack(eval_env, n_stack=4)

In [30]:
train_env.reset()
model = DQN("CnnPolicy", train_env, verbose=0, buffer_size=100000)

with tqdm(total=TRAIN_STEPS) as pbar:
    def callback(_locals, _globals):
        pbar.update(_locals['self'].num_timesteps - pbar.n)
        return True

    model.learn(total_timesteps=TRAIN_STEPS, tb_log_name="first_run", callback=callback)


100%|██████████| 2000000/2000000 [1:51:06<00:00, 300.00it/s]  


In [38]:
model.save("dqn_stable_baselines_2mil_train_steps_100k_buffer_size")

In [39]:
mean_reward, _ = evaluate_policy(model, eval_env, n_eval_episodes=EVAL_EPISODES)
train_env.close()
print(mean_reward)

TypeError: VecFrameStack.reset() got an unexpected keyword argument 'seed'

In [33]:
model = DQN.load('dqn_stable_baselines_2mil_train_steps_100k_buffer_size')

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/save_util.py:437: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  th_object = th.load(file_content, map_locati

In [45]:
#eval_env = gym.make('ALE/Pacman-v5', max_episode_steps=5000, render_mode="rgb_array")
#eval_env = make_atari_env('ALE/Pacman-v5', num_env=1, seed=0)

eval_env = make_atari_env('ALE/Pacman-v5', n_envs=1)
#eval_env = VecFrameStack(eval_env, n_stack=4)

eval_env = RecordVideo(eval_env, 'video')

In [46]:
mean_reward, _ = evaluate_policy(model, eval_env, n_eval_episodes=1)

TypeError: DummyVecEnv.reset() got an unexpected keyword argument 'seed'

In [11]:
print(mean_reward)

48.0


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
